In [ ]:
import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
cfgList=[]
filesDic={}
files2ptKey={}
files3ptKey={}

data={}

def loadData(basePath):
    f={}
    for key in filesDic.keys():
        f[key]={}
        for file in filesDic[key]:
            f[key][file] = h5py.File(basePath + file + '.h5','r')

    for key in files2ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for proj2 in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][proj2]={}
                        for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                            data[key][file][projA][projB][proj2][cont]=np.array([
                                f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                                ])

    for key in files3ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for proj2 in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][proj2]={}
                        for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                            temp_sgn=1
                            # if file in ['B','W','Z']: # for source pion sign missed by PLEGMA
                            #     temp_sgn=-1j 
                            data[key][file][projA][projB][proj2][cont]=np.array([
                                f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                                ])*temp_sgn

    for key in filesDic.keys():
        for file in filesDic[key]:
            f[key][file].close()

basePath_NJN='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJN/proj/'
basePath_NJNpi='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_GEVP/proj/'
basePath_loop='/p/project/pines/li47/code/scratch/run/nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/post_data-merged/'

with h5py.File(basePath_NJN+'NJN.h5') as f:
    cfgList=list(yu.deepKey(f,3).keys())

with h5py.File(basePath_NJNpi+'N_2pt.h5') as f:
    cfgList=list(set(cfgList)&set(yu.deepKey(f,3).keys()))

cfgList.sort()
Ncfg=len(cfgList)
print(Ncfg)


filesDic={}
files2ptKey=[]
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['NJN']
filesDic['NJNpi']=['NJN-pi0i']
loadData(basePath_NJN)

filesDic={}
files2ptKey=['NN','NNpi','NpiNpi']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T','N_2pt-pi0i']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt','N_2pt-pi0f-pi0i','T-pi0f','D1ii-pi0i']
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['N_2pt-j']
filesDic['NJNpi']=['B','W','Z','N_2pt-j-pi0i','N_2pt-j&pi0i','T-j']
loadData(basePath_NJNpi)


data['others']={}
with h5py.File(basePath_loop+'insertLoop.h5') as f:
    data['others']['u+d_id_vev']=np.array([f[cfg]['u+d_id_vev'][()] for cfg in cfgList])
    data['others']['u-d_g5_vev']=np.array([f[cfg]['u-d_g5_vev'][()] for cfg in cfgList])
with h5py.File(basePath_loop+'pi0Loop.h5') as f:
    data['others']['pi0Loop_g5_vev']=np.array([f[cfg]['pi0Loop_g5_vev'][()] for cfg in cfgList])

# <a id='toc1_'></a>[getDat](#toc0_)

In [ ]:
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1

diags_subtract = {'D1ii-pi0i'}

diags_key={}
diags_all=set(); diags_pi0Loop=set(); diags_jLoop=set()
for key in data.keys():
    if key in ['others']:
        continue
    diags_key[key] = set(data[key]) - diags_subtract
    diags_all = diags_all | diags_key[key]
    for diag in data[key]:
        if diag in diags_subtract:
            continue
        t=diag.split('-')
        if 'pi0i' in t or 'pi0f' in t:
            diags_pi0Loop.add(diag)
        if 'j' in t:
            diags_jLoop.add(diag)
diags_loopful = diags_pi0Loop | diags_jLoop
diags_loopless = diags_all - diags_loopful
diags_all_2pt = diags_key['NN'] | diags_key['NNpi'] | diags_key['NpiNpi']
diags_all_3pt = diags_all - diags_all_2pt


Ntime=yu.deepKey(data['NN'],5).shape[1]

def getDat2pt(opa,opb,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['p'] and tpb in ['p']:
        key='NN'
        return data[key]['N_2pt'][proja][projb]['P0'][tpa+'_'+tpb]+np.zeros([Ncfg,Ntime])
    elif tpa in ['p']:
        key='NNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['T']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0i']:
                return [tpa+'_p'] if tpb =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NNpi*np.sum([data[key][dc[0]][proja][projb]['P0'][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])
    elif tpb in ['p']:
        return np.conj(getDat2pt(opb,opa,diags))
    else:
        key='NpiNpi'
        t_diags = diags & (diags_key[key] | diags_subtract)
        def get_conts(diag):
            if diag in ['B_2pt','W_2pt','Z_2pt']:
                return [tpa+'_'+tpb]
            if diag in ['M_correct_2pt']:
                return [tpa+'_'+tpa] if tpa==tpb else []
            if diag in ['N_2pt-pi0f-pi0i']:
                return ['p_p'] if tpa==tpb=='p&pi0' else []
            if diag in ['T-pi0f']:
                return ['p_'+tpb] if tpa=='p&pi0' else []
            if diag in ['D1ii-pi0i']:
                return [tpa+'_p'] if tpb=='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NpiNpi*np.sum([data[key][dc[0]][proja][projb]['P0'][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])

def getDat2ptMat(ops,diags):
    t=np.transpose([[getDat2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def getDat3pt(opa,opb,Gc,iso,proj2,tf,diags):
    (tpa,proja)=opa; (tpb,projb)=opb
    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]
    if tpa in ['p'] and tpb in ['p']:
        key='NJN'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['NJN','N_2pt-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont) for diag in t_diags for cont in get_conts(diag)]
        return np.sum([data[key][dc[0]][proja][projb][proj2][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])       
    elif tpa in ['p']:
        key='NJNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['B','W','Z']:
                return [tpa+'_j'+iso+'_'+tpb+'_deltat_'+str(tf)]
            if diag in ['N_2pt-j&pi0i','N_2pt-j-pi0i','NJN-pi0i']:
                return [tpa+'_p'+'_j'+iso+'_deltat_'+str(tf)] if tpb == 'p&pi0' else []
            if diag in ['T-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NJNpi*np.sum([data[key][dc[0]][proja][projb][proj2][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])
    elif tpb in ['p']:
        if projb[0] != (0,0,0):
            return np.zeros([Ncfg,tf-1])
        return np.conj(getDat3pt(opb,opa,Gc,iso,proj2,tf,diags)[:,::-1])
    else:
        return np.zeros([Ncfg,tf-1])
    
def getDat3ptMat(opas,opbs,Gc,iso,proj2,tf,diags):
    t=np.transpose([[getDat3pt(opa,opb,Gc,iso,proj2,tf,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    return t

tfList=[10,12,14]
def getDat(opas,opbs,Gc,iso,proj2,diags):
    pta=opas[0][1][0]; ptb=opbs[0][1][0]
    assert(pta==(0,0,0)); pc=ptb
    if pc==(0,0,0) and proj2=='Pzt':
        proj2='Pz'
    dat2ptA=getDat2ptMat(opas,diags)
    dat2ptB=getDat2ptMat(opbs,diags)
    dat3pt={}
    for tf in tfList:
        dat3pt[tf]=getDat3ptMat(opas,opbs,Gc,iso,proj2,tf,diags)
    if (Gc,iso) == ('id','+'):
        dat_jLoop=data['others']['u+d_id_vev']
    elif (Gc,iso) == ('g5','-'):
        dat_jLoop=data['others']['u-d_g5_vev']
    else:
        dat_jLoop=np.zeros(Ncfg)

    t=np.array([
        np.array([dat2ptA[cfg],dat2ptB[cfg]]+[np.array([dat3pt[tf][cfg] for tf in tfList],dtype=object)]+[dat_jLoop[cfg]],dtype=object)
    for cfg in range(Ncfg)], dtype=object)
    return t

# # test
# pt1=(0,0,0); pt2=(0,0,1)
# proj1=(pt1,'a'); proj12=(pt2,'a')
# proj2=(pt1,('N1\\pi1','a')); proj22=(pt2,('N1\\pi0','a'))
# op1=('p',proj1); op12=('p',proj12)
# op2=('n&pi+',proj2); op22=('n&pi+',proj22)
# op3=('p&pi0',proj2); op32=('p&pi0',proj22)

# getDat3pt(op1,op32,'id','+','Pzt',10,diags_all).shape
# t=getDat([op1,op2,op3],[op12,op22,op32],'id','+','Pzt',10,diags_all)

# <a id='toc2_'></a>[2pt](#toc0_)

In [ ]:
def run(opa,opb,diags,title,ylim1=None,ylim2=[.4,1.1],preFactor=1):
    tmin=1; tmax=15

    (Nrow, Ncol) = (1, 2)
    (Lrow, Lcol) = (4, 5)
    fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False)
    fig.suptitle(title)

    dat=preFactor*getDat2pt(opa,opb,diags)

    irow=0; icol=0
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        t=np.real(t)
        return t
    (mean,cov,err) = yu.jackknife(dat,tFunc)
    tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color='r',fmt='s',elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(0,tmax,2))
    axs[irow,icol].set_ylim(ylim1)

    irow=0; icol=1
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        t=np.real(t)
        t=np.log(t/np.roll(t,-1,axis=0))
        return t
    (mean,cov,err) = yu.jackknife(dat,tFunc)
    tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color='r',fmt='s',elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(0,tmax,2))
    axs[irow,icol].set_ylim(ylim2)
    if opa[1][0] == (0,0,0):
        axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
        axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
    elif opa[1][0] == (0,0,1):
        axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
        axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan')
        axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange')

## <a id='toc2_1_'></a>[pt=(0,0,0)](#toc0_)

In [ ]:
pt=(0,0,0)
op=('p',(pt,'a')); diags=diags_all
run(op,op,diags,'p_p')

In [ ]:
pt=(0,0,0)
proj1=(pt,'a'); proj2=(pt,('N1\\pi1','a'))
opa=('p',proj1); opb=('n&pi+',proj2)

run(opa,opb,diags,'p_n&pi+: real')
run(opa,opb,diags,'p_n&pi+: imag',preFactor=-1j)

opa=('p',proj1); opb=('p&pi0',proj2)
run(opa,opb,diags,'p_p&pi0: real')
run(opa,opb,diags,'p_p&pi0: imag',preFactor=-1j)
run(opa,opb,diags_loopless,'p_p&pi0: imag (loopless)',preFactor=-1j)

why purely imaginary?

In [ ]:
pt=(0,0,0)
proj=(pt,('N1\\pi1','a'))
op=('n&pi+',proj)

diags=diags_all
run(op,op,diags,'n&pi+_n&pi+')

diags={'M_correct_2pt'}
run(op,op,diags,'n&pi+_n&pi+: M')

diags={'B_2pt','W_2pt','Z_2pt'}
run(op,op,diags,'n&pi+_n&pi+: BWZ')

In [ ]:
pt=(0,0,0)
proj=(pt,('N1\\pi1','a'))
op=('p&pi0',proj)

diags=diags_all | diags_subtract
print(diags)
run(op,op,diags,'p&pi0_p&pi0: with D1ii-pi0i')

diags=diags_all
run(op,op,diags,'p&pi0_p&pi0')

diags=diags_loopless
run(op,op,diags,'p&pi0_p&pi0: loopless')

diags=diags_loopful
run(op,op,diags,'p&pi0_p&pi0: loopful')

diags={'D1ii-pi0i'}
run(op,op,diags,'p&pi0_p&pi0: D1ii-pi0i only')

diags={'T-pi0f'}
run(op,op,diags,'p&pi0_p&pi0: T-pi0f only')

In [ ]:
pt=(0,0,0)
proj=(pt,('N1\\pi1','a'))
opa=('n&pi+',proj); opb=('p&pi0',proj)

print(diags_key['NpiNpi']) 
diags=diags_all-{'W_2pt','T-pi0f'}
diags={'W_2pt'}
run(opa,opb,diags,'n&pi+_p&pi0: W')
run(opb,opa,diags,'p&pi0_n&pi+: W')

differ by a factor?

In [ ]:
print(3/1.75)
print(1.7/0.9)
print(1.0/0.5)

## <a id='toc2_2_'></a>[pt=(0,0,1)](#toc0_)

In [ ]:
pt=(0,0,1)
op=('p',(pt,'a')); diags=diags_all
run(op,op,diags,'p_p')

In [ ]:
pt=(0,0,1)
proj1=(pt,'a'); proj2=(pt,('N1\\pi0','a'))
opa=('p',proj1); opb=('n&pi+',proj2)

run(opa,opb,diags,'p_n&pi+: real')
run(opa,opb,diags,'p_n&pi+: imag',preFactor=-1j)

opa=('p',proj1); opb=('p&pi0',proj2)
run(opa,opb,diags,'p_p&pi0: real')
run(opa,opb,diags,'p_p&pi0: imag',preFactor=-1j)
run(opa,opb,diags_loopless,'p_p&pi0: imag (loopless)',preFactor=-1j)

In [ ]:
data['others']['pi0Loop_g5_vev'][:6]

In [ ]:
run(opa,opb,{'N_2pt-pi0i'},'p_p&pi0: real',preFactor=1)
run(opa,opb,{'N_2pt-pi0i'},'p_p&pi0: real',preFactor=-1j)

p(0,01)_p(0,0,1)&pi0(0,0,0): p(0,01)_p(0,0,1) * pi0(0,0,0)

should be larger

In [ ]:
pt=(0,0,1)
proj=(pt,('N1\\pi0','a'))
op=('n&pi+',proj)

diags=diags_all
run(op,op,diags,'n&pi+_n&pi+')

diags={'M_correct_2pt'}
run(op,op,diags,'n&pi+_n&pi+: M')

diags={'B_2pt','W_2pt','Z_2pt'}
run(op,op,diags,'n&pi+_n&pi+: BWZ')

In [ ]:
pt=(0,0,1)
proj=(pt,('N1\\pi0','a'))

print(diags_pi0Loop)

diags=diags_all
run(op,op,diags,'p&pi0_p&pi0: with D1ii-pi0i')

diags=diags_all
run(op,op,diags,'p&pi0_p&pi0')

diags=diags_loopless
run(op,op,diags,'p&pi0_p&pi0: loopless')

diags=diags_loopful
run(op,op,diags,'p&pi0_p&pi0: loopful')

diags={'D1ii-pi0i'}
run(op,op,diags,'p&pi0_p&pi0: D1ii-pi0i only')

diags={'T-pi0f'}
run(op,op,diags,'p&pi0_p&pi0: T-pi0f only')

In [ ]:
def run(ops,diags,title):
    N_op=len(ops); tRef=4

    (Nrow, Ncol) = (1, 2)
    fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
    fig.suptitle(title)
    yu.addColHeader(axs,['Effective mass from E-values','Ratio from E-vectors'])
    colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange',5:'cyan',6:'pink'}
    fmtDic={0:'s',1:'d',2:'o'}

    tmin=1; tmax=15

    dat=getDat2ptMat(ops,diags)

    irow=0; icol=0
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        (eVals,eVecs)=yu.GEVP(t,tRef)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))
        return En.T
    
    (mean,cov,err)=yu.jackknife(dat,tFunc,2)
    
    for i in range(N_op):
        ttmax=tmax
        tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
        axs[irow,icol].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt='s',color=colorDic[i],elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(0,tmax,2))
    axs[irow,icol].set_ylim(.4,1.4)

    if ops[0][1][0] == (0,0,0):
        axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
        axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
    elif ops[0][1][0] == (0,0,1):
        axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
        axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 

    icol=1
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        (eVals,eVecs)=yu.GEVP(t,tRef)
        t=eVecs[:,:,0].T
        return np.abs(t)
        return np.array([np.real(t[0]),np.imag(t[1]),np.imag(t[2])])
    (mean,cov,err)=yu.jackknife(dat,tFunc,2)
    mean0=mean[0,tmin:tmax]

    for i in range(N_op):
        ttmax=tmax
        tMean=mean[i,tmin:ttmax]/mean0;tErr=err[i,tmin:ttmax]/mean0

        # temp_min=8
        # t_factor=1 if i!=1 else 1/np.sqrt(2)
        # print(i)
        # print(yu.npRound(tMean[temp_min:]*t_factor,3))
        # print(yu.npRound(tErr[temp_min:]*t_factor,3))

        axs[irow,icol].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt=fmtDic[i],color='r',elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(0,tmax,2))
    axs[irow,icol].set_ylim(1e-4,1.1)
    axs[irow,icol].set_yscale('log')
    axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])

In [ ]:
diags=diags_all
pt=(0,0,0)
proj1=(pt,'a'); proj2=(pt,('N1\\pi1','a')); proj3 = (pt,('N2\\pi2','a'));
ops=[('p',proj1),('n&pi+',proj2)]
run(ops,diags,'op1=p; op2=n&pi+')

ops=[('p',proj1),('p&pi0',proj2)]
run(ops,diags,'op1=p; op2=p&pi0')

ops=[('p',proj1),('p&pi0',proj2)]
run(ops,diags_loopless,'op1=p; op2=p&pi0: loopless')

ops=[('p',proj1),('n&pi+',proj2),('p&pi0',proj2)]
run(ops,diags,'op1=p; op2=n&pi+; op3=p&pi0')

In [ ]:
diags=diags_all
pt=(0,0,0)
proj1=(pt,'a'); proj2=(pt,('N1\\pi1','a'))
ops=[('p',proj1),('p&pi0',proj2)]
ops=[('p',proj1),('n&pi+',proj2)]


dat=getDat2ptMat(ops,diags)

tRef=4

t=np.mean(dat,axis=0)

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     (eVals,eVecs)=yu.GEVP(t,tRef)
#     En=np.log(eVals/np.roll(eVals,-1,axis=0))
#     return En.T

# (mean,cov,err)=yu.jackknife(dat,tFunc,2)

print(t[4])
print(t[12])

(eVals,eVecs)=yu.GEVP(t,4)
print(eVals[12])
print(eVecs[12])

In [ ]:
diags=diags_all
pt=(0,0,1)
proj1=(pt,'a'); proj2=(pt,('N1\\pi0','a'))
ops=[('p',proj1),('n&pi+',proj2)]
run(ops,diags,'op1=p; op2=n&pi+')

ops=[('p',proj1),('p&pi0',proj2)]
run(ops,diags,'op1=p; op2=p&pi0')

ops=[('p',proj1),('p&pi0',proj2)]
run(ops,diags_loopless,'op1=p; op2=p&pi0: loopless')

ops=[('p',proj1),('n&pi+',proj2),('p&pi0',proj2)]
run(ops,diags,'op1=p; op2=n&pi+; op3=p&pi0')

In [ ]:
def run(opa,opb,Gc,iso,proj2,diags,title,preFactor=1,ylim1=None,ylim2=None):
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    (Nrow, Ncol) = (1, 2)
    (Lrow, Lcol) = (4, 5)
    fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False)
    fig.suptitle('(Gc,iso,proj2,preFactor)='+str((Gc,iso,proj2,preFactorStr))+': '+title)
    yu.addColHeader(axs,['3pt','3pt/2pt'])
    
    dat=getDat([opa],[opb],Gc,iso,proj2,diags)

    vevFactor = 1 if 'N_2pt-j' in diags and opa==opb else 0

    irow=0; icol=0
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        t=np.array([np.real(preFactor*(t[2][i_tf][:,0,0]-vevFactor*t[0][tfList[i_tf],0,0]*t[3])) for i_tf in range(len(tfList))],dtype=object)
        return t

    (mean,cov,err)=yu.jackknife(dat,tFunc,2)
    
    colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
    fmtDic={10:'s',12:'d',14:'o'}

    for i_tf in range(len(tfList)):
        tf=tfList[i_tf]
        tMean=mean[i_tf];tErr=err[i_tf]
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
            fmt=fmtDic[tf],color=colorDic[(0,0)],elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(-6,6,2))
    axs[irow,icol].set_ylim(ylim1)

    irow=0; icol=1
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        
        def getRes(i_tf):
            tf=tfList[i_tf]
            nom=t[2][i_tf][:,0,0]
            denom=np.sqrt( np.real(t[0][tf]) * np.real(t[1][tf]) *\
            np.real(t[0][1:tf])[::-1]/np.real(t[0][1:tf]) *\
            np.real(t[1][1:tf]/np.real(t[1][1:tf])[::-1]) )[:,0,0]
            return np.real(preFactor * (nom/denom-vevFactor*t[3]) )
        t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
        return t

    (mean,cov,err)=yu.jackknife(dat,tFunc,2)
    
    colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
    fmtDic={10:'s',12:'d',14:'o'}

    for i_tf in range(len(tfList)):
        tf=tfList[i_tf]
        tMean=mean[i_tf];tErr=err[i_tf]
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
            fmt=fmtDic[tf],color=colorDic[(0,0)],elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(-6,6,2))
    axs[irow,icol].set_ylim(ylim2)

pt1=(0,0,0); pt2=(0,0,1)
proj1=(pt1,'a'); proj12=(pt2,'a')
proj2=(pt1,('N1\\pi1','a')); proj22=(pt2,('N1\\pi0','a')); proj23=(pt2,('N0\\pi1','a'))
op1=('p',proj1); op12=('p',proj12)
op2=('n&pi+',proj2); op22=('n&pi+',proj22); op23=('n&pi+',proj23)
op3=('p&pi0',proj2); op32=('p&pi0',proj22); op33=('p&pi0',proj23)

In [ ]:
opa=op1; opb=op1
(Gc,id,proj2,preFactor)=('id','+','P0',1)
ylim1=None; ylim2=None
run(opa,opb,Gc,id,proj2,diags_all,'full',preFactor,ylim1=ylim1,ylim2=ylim2)
run(opa,opb,Gc,id,proj2,diags_loopless,'loopless',preFactor,ylim1=ylim1,ylim2=ylim2)
run(opa,opb,Gc,id,proj2,diags_loopful,'loopful',preFactor,ylim1=ylim1,ylim2=ylim2)

In [ ]:
opa=op1; opb=op2
(Gc,id,proj2,preFactor)=('id','+','P0',1)
ylim1=None; ylim2=None
# run(opa,opb,Gc,id,proj2,diags_all,'full',preFactor,ylim1=ylim1,ylim2=ylim2)
# run(opa,opb,Gc,id,proj2,diags_loopless,'loopless',preFactor,ylim1=ylim1,ylim2=ylim2)
# run(opa,opb,Gc,id,proj2,diags_loopful,'loopful',preFactor,ylim1=ylim1,ylim2=ylim2)        

run(opa,opb,Gc,id,proj2,diags_loopless,'full',preFactor,ylim1=ylim1,ylim2=ylim2)
run(opa,opb,Gc,id,proj2,{'B','W','Z'},'full',preFactor,ylim1=ylim1,ylim2=ylim2)

In [ ]:
opa=op1; opb=op33
(Gc,id,proj2,preFactor)=('g5','-','Pz',-1j)
ylim1=None; ylim2=None
# run(opa,opb,Gc,id,proj2,diags_all,'conn+disc',preFactor,ylim1=ylim1,ylim2=ylim2)
# run(opa,opb,Gc,id,proj2,diags_loopless,'conn',preFactor,ylim1=ylim1,ylim2=ylim2)
# run(opa,opb,Gc,id,proj2,diags_loopful,'disc',preFactor,ylim1=ylim1,ylim2=ylim2)

run(opa,opb,Gc,id,proj2,{'N_2pt-j&pi0i'},'disc',preFactor,ylim1=ylim1,ylim2=ylim2)
run(opa,opb,Gc,id,proj2,{'N_2pt-j-pi0i'},'disc',preFactor,ylim1=ylim1,ylim2=ylim2)

In [ ]:
def run(opas,opbs,Gc,iso,proj2,diags,title,preFactor=1,ylim1=None,ylim2=None):
    tRef=1
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    (Nrow, Ncol) = (1, 2)
    (Lrow, Lcol) = (4, 5)
    fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False)
    fig.suptitle('(Gc,iso,proj2,preFactor)='+str((Gc,iso,proj2,preFactorStr))+': '+title)
    yu.addColHeader(axs,['3pt/2pt','3pt/2pt GEVP improved'])
    
    vevFactor = 1 if 'N_2pt-j' in diags and opas[0]==opbs[0] else 0

    irow=0; icol=0
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
        (eValsb,eVecsb)=yu.GEVP(t[1],tRef)
        def getRes(i_tf):
            tf=tfList[i_tf]
            eVeca=eVecsa[tf,:,0]
            eVecb=eVecsb[tf,:,0]
            nom=np.conj(eVeca).T@t[2][i_tf]@eVecb
            denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
            np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
            np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
            return np.real(preFactor * (nom/denom-vevFactor*t[3]) )
            
        t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
        return t

    dat=getDat(opas[:1],opbs[:1],Gc,iso,proj2,diags)
    (mean,cov,err)=yu.jackknife(dat,tFunc,2)
    
    colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
    fmtDic={10:'s',12:'d',14:'o'}

    for i_tf in range(len(tfList)):
        tf=tfList[i_tf]
        tMean=mean[i_tf];tErr=err[i_tf]
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
            fmt=fmtDic[tf],color=colorDic[(0,0)],elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(-6,6,2))
    axs[irow,icol].set_ylim(ylim1)

    icol=1
    dat=getDat(opas,opbs,Gc,iso,proj2,diags)
    (mean,cov,err)=yu.jackknife(dat,tFunc,2)
    
    colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
    fmtDic={10:'s',12:'d',14:'o'}

    for i_tf in range(len(tfList)):
        tf=tfList[i_tf]
        tMean=mean[i_tf];tErr=err[i_tf]
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
            fmt=fmtDic[tf],color=colorDic[(0,0)],elinewidth=2,capsize=6,mfc='white')
    axs[irow,icol].grid()
    axs[irow,icol].set_xticks(range(-6,6,2))
    axs[irow,icol].set_ylim(ylim2)

pt1=(0,0,0); pt2=(0,0,1)
proj1=(pt1,'a'); proj12=(pt2,'a')
proj2=(pt1,('N1\\pi1','a')); proj22=(pt2,('N1\\pi0','a')); proj23=(pt2,('N0\\pi1','a'))
op1=('p',proj1); op12=('p',proj12)
op2=('n&pi+',proj2); op22=('n&pi+',proj22); op23=('n&pi+',proj23)
op3=('p&pi0',proj2); op32=('p&pi0',proj22); op33=('p&pi0',proj23)

In [ ]:
opas=[op1,op2]; opbs=opas
(Gc,id,proj2,preFactor)=('id','+','P0',1)
ylim1=[4,16]; ylim2=ylim1

test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run(opas,opbs,Gc,id,proj2,diags_loopless,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1.2
run(opas,opbs,Gc,id,proj2,diags_loopless,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1

In [ ]:
opas=[op1,op2]; opbs=opas
(Gc,id,proj2,preFactor)=('id','+','P0',1)
ylim1=[4,16]; ylim2=ylim1

test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run(opas,opbs,Gc,id,proj2,diags_all,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1.2
run(opas,opbs,Gc,id,proj2,diags_all,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1

In [ ]:
diags_all_2pt

In [ ]:
opas=[op1,op2]; opbs=opas
(Gc,id,proj2,preFactor)=('id','+','P0',1)
ylim1=[0,20]; ylim2=ylim1

run(opas,opbs,Gc,id,proj2,diags_all-{'D1ii-pi0i'},'',preFactor,ylim1=ylim1,ylim2=ylim2)
print(diags_jLoop)
# run(opas,opbs,Gc,id,proj2,(diags_all_2pt-'D1ii-pi0i') & diags_jLoop,'',preFactor,ylim1=ylim1,ylim2=ylim2)

In [ ]:
opas=[op1,op3]; opbs=opas
(Gc,id,proj2,preFactor)=('id','+','P0',1)
ylim1=[5,20]; ylim2=ylim1

test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run(opas,opbs,Gc,id,proj2,diags_loopless,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1.2
run(opas,opbs,Gc,id,proj2,diags_loopless,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1

In [ ]:
opas=[op1,op3]; opbs=opas
(Gc,id,proj2,preFactor)=('id','+','P0',1)
ylim1=[5,20]; ylim2=ylim1

test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run(opas,opbs,Gc,id,proj2,diags_all,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1.2
run(opas,opbs,Gc,id,proj2,diags_all,'',preFactor,ylim1=ylim1,ylim2=ylim2)
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1